In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import copy

In [2]:
from PIL import Image

In [3]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D

Using TensorFlow backend.


In [4]:
from keras import backend as K
K.set_image_data_format( 'channels_last' )

In [5]:
def convblock(cdim, nb, bits=3):  #define repeating layers og VGGNet at a block
    L = []
    
    for k in range(1,bits+1):
        convname = 'conv'+str(nb)+'_'+str(k)
        #L.append( Convolution2D(cdim, 3, 3, border_mode='same', activation='relu', name=convname) ) # Keras 1
        L.append( Convolution2D(cdim, kernel_size=(3, 3), padding='same', activation='relu', name=convname) ) # Keras 2
    
    L.append( MaxPooling2D((2, 2), strides=(2, 2)) )
    
    return L

In [6]:
def vgg_face_blank(): #define VGGFace model
    
    withDO = True 
    
    if True:
        mdl = Sequential()
        
        # First layer is a dummy-permutation = Identity to specify input shape
        mdl.add( Permute((1,2,3), input_shape=(224,224,3)) ) #0 is the sample dim

        for l in convblock(64, 1, bits=2):
            mdl.add(l)

        for l in convblock(128, 2, bits=2):
            mdl.add(l)
        
        for l in convblock(256, 3, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 4, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 5, bits=3):
            mdl.add(l)
        
        
        mdl.add( Convolution2D(4096, kernel_size=(7, 7), activation='relu', name='fc6') ) # Keras 2
        if withDO:
            mdl.add( Dropout(0.5) )
       
        mdl.add( Convolution2D(4096, kernel_size=(1, 1), activation='relu', name='fc7') ) # Keras 2
        if withDO:
            mdl.add( Dropout(0.5) )
      
        mdl.add( Convolution2D(2622, kernel_size=(1, 1), activation='relu', name='fc8') ) # Keras 2
        mdl.add( Flatten() )
        mdl.add( Activation('softmax') )
        
        return mdl
    
    else:
        raise ValueError('not implemented')

In [7]:
facemodel = vgg_face_blank() #define model






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
#facemodel.summary()

In [8]:
from scipy.io import loadmat #import weights

In [9]:
    data = loadmat('C:\\Users\\Om\\Desktop\\vgg-face.mat', matlab_compatible=False, struct_as_record=False)
    l = data['layers']
    description = data['meta'][0,0].classes[0,0].description

In [10]:
#l.shape, description.shape

In [11]:
#l[0,10][0,0].type[0], l[0,10][0,0].name[0]

In [12]:
def weight_compare(kmodel): #convert weights from MATLAB to python format
    kerasnames = [lr.name for lr in kmodel.layers]
    prmt = (0,1,2,3) 

    for i in range(l.shape[1]):
        matname = l[0,i][0,0].name[0]
        mattype = l[0,i][0,0].type[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            print(matname, mattype)
            print(l[0,i][0,0].weights[0,0].transpose(prmt).shape, l[0,i][0,0].weights[0,1].shape)
            print(kmodel.layers[kindex].get_weights()[0].shape, kmodel.layers[kindex].get_weights()[1].shape)
            print('------------------------------------------')
        else:
            print('MISSING : ', matname, mattype)
            print('------------------------------------------')

In [13]:
def copy_mat_to_keras(kmodel):

    kerasnames = [lr.name for lr in kmodel.layers]
    prmt = (0,1,2,3)

    for i in range(l.shape[1]):
        matname = l[0,i][0,0].name[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            
            l_weights = l[0,i][0,0].weights[0,0]
            l_bias = l[0,i][0,0].weights[0,1]
            f_l_weights = l_weights.transpose(prmt)
            assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
            assert (l_bias.shape[1] == 1)
            assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
            assert (len(kmodel.layers[kindex].get_weights()) == 2)
            kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])

In [14]:
copy_mat_to_keras(facemodel) #load weights into model

In [15]:
featuremodel = Model(inputs=facemodel.layers[0].input, outputs=facemodel.layers[-1].output) #drop softmax layer

### Image cropping for input

In [16]:
#plt.imshow(np.asarray(im))

def pred(kmodel, crpimg, transform=False):
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940
        #
        # WARNING : in this script (https://github.com/rcmalli/keras-vggface) colours are switched
        aux = copy.copy(imarr)
        #imarr[:, :, 0] = aux[:, :, 2]
        #imarr[:, :, 2] = aux[:, :, 0]

        #imarr[:,:,0] -= 129.1863
        #imarr[:,:,1] -= 104.7624
        #imarr[:,:,2] -= 93.5940

    #imarr = imarr.transpose((2,0,1)) # INFO : for 'th' setting of 'dim_ordering'
    imarr = np.expand_dims(imarr, axis=0)

    out = kmodel.predict(imarr)

    best_index = np.argmax(out, axis=1)[0]
    best_name = description[best_index,0]
    print(best_index, best_name[0], out[0,best_index], [np.min(out), np.max(out)])

In [17]:
def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)

    fvec = featmodel.predict(imarr)[0,:]
    # normalize
    normfvec = math.sqrt(fvec.dot(fvec))
    return fvec/normfvec

In [18]:
import os

### extracting face features of positive examples

In [21]:
sample_space=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[0]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[0]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
for utterance in lwordFolder:
    j += 1
    utterFolder= '%s\\%s' % (wordFolder,utterance)
    lutterFolder = os.listdir(utterFolder)
    #print(lutterFolder)
    colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
    lcolourFolder = os.listdir(colourFolder)
    i=0
    for frame in lcolourFolder:
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(featuremodel,im, transform=True)
        #print(feature_vector.shape)
        feature_vector= feature_vector.reshape((1,len(feature_vector)))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector))
    print(j)
    if j == 1:
        sample_space=feature_sequence
    else:
        sample_space=np.concatenate((sample_space,feature_sequence))
    if j==9:
        break

1
2
3
4
5
6
7
8
9


In [22]:
#print(sample_space.shape)
sample_space= sample_space.reshape(9,len(lcolourFolder),feature_vector.shape[1]) #input dataset 
print(sample_space.shape[1])
pos_sample_space=sample_space
print(pos_sample_space)

11
[[[6.75963690e-07 8.25880124e-06 1.11793170e-05 ... 4.66088778e-07
   5.59341206e-05 1.89274695e-04]
  [3.68687834e-05 3.32745840e-05 2.26098364e-05 ... 2.38872030e-06
   4.72466258e-04 2.51124118e-04]
  [2.25483982e-05 6.61253989e-06 5.48863463e-05 ... 2.88070873e-06
   1.88437087e-04 1.87333702e-04]
  ...
  [7.30168085e-06 7.31176397e-06 3.40432707e-05 ... 9.35737035e-07
   1.06274128e-04 1.38831048e-04]
  [2.87230250e-05 4.04849852e-05 1.27139632e-04 ... 6.01278862e-06
   4.72722022e-04 4.73145512e-04]
  [4.25673313e-02 6.09404147e-02 1.02217663e-02 ... 6.68874290e-03
   2.12680474e-02 2.88120322e-02]]

 [[1.21957986e-04 4.71811436e-05 3.15443234e-04 ... 9.53527524e-06
   5.55832498e-03 1.44382613e-03]
  [1.36644085e-04 5.08134399e-05 3.36446741e-04 ... 1.32323767e-05
   3.96228861e-04 4.30966448e-03]
  [1.29346838e-04 2.83445770e-05 6.78209937e-04 ... 7.15792976e-06
   1.15853458e-04 2.36908044e-03]
  ...
  [4.25673313e-02 6.09404147e-02 1.02217663e-02 ... 6.68874290e-03
   2.12

In [23]:
sample_space_1=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[0]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[1]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
for utterance in lwordFolder:
    j += 1
    utterFolder= '%s\\%s' % (wordFolder,utterance)
    lutterFolder = os.listdir(utterFolder)
    #print(lutterFolder)
    colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
    lcolourFolder = os.listdir(colourFolder)
    i=0
    for frame in lcolourFolder:
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(featuremodel,im, transform=True)
        #print(feature_vector.shape)
        feature_vector= feature_vector.reshape((1,len(feature_vector)))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector))
    print(j)
    if j == 1:
        sample_space_1=feature_sequence
    else:
        sample_space_1=np.concatenate((sample_space_1,feature_sequence))
    if j==9:
        break

1
2
3
4
5
6
7
8
9


In [24]:
sample_space_2= sample_space_1.reshape(9,len(lcolourFolder),feature_vector.shape[1])
print(sample_space_2.shape)
sample_space_2=np.concatenate((sample_space_2,np.zeros((9,3,2622))),axis=1)
print(sample_space_2.shape)
total_sample_space=np.concatenate((pos_sample_space,sample_space_2))
print(total_sample_space.shape)

(9, 8, 2622)
(9, 11, 2622)
(18, 11, 2622)


In [25]:
sample_space_3=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[6]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[0]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
for utterance in lwordFolder:
    j += 1
    utterFolder= '%s\\%s' % (wordFolder,utterance)
    lutterFolder = os.listdir(utterFolder)
    #print(lutterFolder)
    colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
    lcolourFolder = os.listdir(colourFolder)
    i=0
    for frame in lcolourFolder:
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(featuremodel,im, transform=True)
        #print(feature_vector.shape)
        feature_vector= feature_vector.reshape((1,len(feature_vector)))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector))
    print(j)
    if j == 1:
        sample_space_3=feature_sequence
    else:
        sample_space_3=np.concatenate((sample_space_3,feature_sequence))
    if j==9:
        break

1
2
3
4
5
6
7
8
9


In [27]:
sample_space_4= sample_space_3.reshape(9,len(lcolourFolder),feature_vector.shape[1])
print(sample_space_4.shape)
total_sample_space1=np.concatenate((total_sample_space,sample_space_4))
print(total_sample_space1.shape)

(9, 11, 2622)
(27, 11, 2622)


In [35]:
sample_space_5=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[10]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[2]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
for utterance in lwordFolder:
    j += 1
    utterFolder= '%s\\%s' % (wordFolder,utterance)
    lutterFolder = os.listdir(utterFolder)
    #print(lutterFolder)
    colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
    lcolourFolder = os.listdir(colourFolder)
    i=0
    for frame in lcolourFolder:
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(featuremodel,im, transform=True)
        #print(feature_vector.shape)
        feature_vector= feature_vector.reshape((1,len(feature_vector)))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector))
    print(j)
    if j == 1:
        sample_space_5=feature_sequence
    else:
        sample_space_5=np.concatenate((sample_space_5,feature_sequence))
    if j==9:
        break

1
2
3
4
5
6
7
8
9


In [36]:
print(sample_space_5.shape)
sample_space_6= sample_space_5.reshape(9,len(lcolourFolder),feature_vector.shape[1])
print(sample_space_6.shape)
total_sample_space2=np.concatenate((total_sample_space1,sample_space_6))
print(total_sample_space2.shape)

(99, 2622)
(9, 11, 2622)
(36, 11, 2622)


In [19]:
from numpy import save

In [39]:
save('x_train.npy',total_sample_space2)

In [90]:
#import into 
y_train_1=np.ones((9,1,1))
y_train_2=np.zeros((9,1,1))
y_train=np.concatenate((y_train_1,y_train_2))
print(y_train.shape)


(18, 1, 1)


### LSTM Layers

In [2]:
from keras.layers import LSTM

Using TensorFlow backend.


In [ ]:
model = Sequential()
input_shape=(total_sample_space.shape[1],total_sample_space.shape[2])
model.add(LSTM(units=10,return_sequences=True,input_shape=input_shape))
model.add(Dropout(0.2))

model.add(LSTM(units=10,return_sequences=True,input_shape=input_shape))
model.add(Dropout(0.2))

model.add(LSTM(units=10,return_sequences=True,input_shape=input_shape))
model.add(Dropout(0.2))

model.add(LSTM(units=10,input_shape=input_shape))
model.add(Dropout(0.2))

model.add(Dense(units=1))



In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(total_sample_space,y_train,epochs=3,batch_size=9)

In [3]:
model.predict_proba(feature_sequence_1)

NameError: name 'model' is not defined

In [20]:
sample_space=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[0]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[0]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
j += 1
utterFolder= '%s\\%s' % (wordFolder,lwordFolder[9])
lutterFolder = os.listdir(utterFolder)
#print(lutterFolder)
colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
lcolourFolder = os.listdir(colourFolder)
i=0
for frame in lcolourFolder:
    i += 1
    image= "%s\\%s" % (colourFolder,frame) 
    im=Image.open(image)
    im = im.resize((224,224))
    feature_vector = features(featuremodel,im, transform=True)
    #print(feature_vector.shape)
    feature_vector= feature_vector.reshape((1,len(feature_vector)))
    if i==1 :
        feature_sequence=feature_vector #done because of need of same dim. for concatenation
    else:
        feature_sequence = np.concatenate((feature_sequence,feature_vector))
print(j)

1


In [21]:
x_test1 = feature_sequence.reshape((1,11,2622))

In [22]:
sample_space=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[0]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[1]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
j += 1
utterFolder= '%s\\%s' % (wordFolder,lwordFolder[9])
lutterFolder = os.listdir(utterFolder)
#print(lutterFolder)
colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
lcolourFolder = os.listdir(colourFolder)
i=0
for frame in lcolourFolder:
    i += 1
    image= "%s\\%s" % (colourFolder,frame) 
    im=Image.open(image)
    im = im.resize((224,224))
    feature_vector = features(featuremodel,im, transform=True)
    #print(feature_vector.shape)
    feature_vector= feature_vector.reshape((1,len(feature_vector)))
    if i==1 :
        feature_sequence=feature_vector #done because of need of same dim. for concatenation
    else:
        feature_sequence = np.concatenate((feature_sequence,feature_vector))
print(j)

1


In [23]:
x_test2 = feature_sequence.reshape((1,11,2622))
x_test3 = np.concatenate((x_test1,x_test2))

In [24]:
sample_space=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[6]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[0]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
j += 1
utterFolder= '%s\\%s' % (wordFolder,lwordFolder[9])
lutterFolder = os.listdir(utterFolder)
#print(lutterFolder)
colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
lcolourFolder = os.listdir(colourFolder)
i=0
for frame in lcolourFolder:
    i += 1
    image= "%s\\%s" % (colourFolder,frame) 
    im=Image.open(image)
    im = im.resize((224,224))
    feature_vector = features(featuremodel,im, transform=True)
    #print(feature_vector.shape)
    feature_vector= feature_vector.reshape((1,len(feature_vector)))
    if i==1 :
        feature_sequence=feature_vector #done because of need of same dim. for concatenation
    else:
        feature_sequence = np.concatenate((feature_sequence,feature_vector))
print(j)

1


In [25]:
x_test4 = feature_sequence.reshape((1,11,2622))
x_test5 = np.concatenate((x_test3,x_test4))

In [26]:
sample_space=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[10]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[2]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
j += 1
utterFolder= '%s\\%s' % (wordFolder,lwordFolder[9])
lutterFolder = os.listdir(utterFolder)
#print(lutterFolder)
colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
lcolourFolder = os.listdir(colourFolder)
i=0
for frame in lcolourFolder:
    i += 1
    image= "%s\\%s" % (colourFolder,frame) 
    im=Image.open(image)
    im = im.resize((224,224))
    feature_vector = features(featuremodel,im, transform=True)
    #print(feature_vector.shape)
    feature_vector= feature_vector.reshape((1,len(feature_vector)))
    if i==1 :
        feature_sequence=feature_vector #done because of need of same dim. for concatenation
    else:
        feature_sequence = np.concatenate((feature_sequence,feature_vector))
print(j)

1


In [29]:
x_test6 = feature_sequence.reshape((1,11,2622))
x_test7 = np.concatenate((x_test5,x_test6))


In [30]:
x_test7.shape

(4, 11, 2622)

In [20]:
sample_space=[] #for appending different utterances
feature_sequence = [] # for appending different timesteps
inpDir='C:\\Users\\Om\\Desktop\\Dataset' #input dataset
person = {}
linpDir = os.listdir(inpDir) #list all directories in dataset
#print(linpDir)
personStr= linpDir[0]
personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
lpersonFolder = os.listdir(personFolder)
#print(lpersonFolder)
wordID= lpersonFolder[3]
wordFolder = '%s\\%s' % (personFolder,wordID)
lwordFolder = os.listdir(wordFolder)
#print(lwordFolder)
j=0
j += 1
utterFolder= '%s\\%s' % (wordFolder,lwordFolder[0])
lutterFolder = os.listdir(utterFolder)
#print(lutterFolder)
colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
lcolourFolder = os.listdir(colourFolder)
i=0
for frame in lcolourFolder:
    i += 1
    image= "%s\\%s" % (colourFolder,frame) 
    im=Image.open(image)
    im = im.resize((224,224))
    feature_vector = features(featuremodel,im, transform=True)
    #print(feature_vector.shape)
    feature_vector= feature_vector.reshape((1,len(feature_vector)))
    if i==1 :
        feature_sequence=feature_vector #done because of need of same dim. for concatenation
    else:
        feature_sequence = np.concatenate((feature_sequence,feature_vector))
print(j)

1


In [21]:
x_test8=feature_sequence.reshape((1,11,2622))

In [22]:
save('x_test_4.npy',x_test8)